 <h2 align="center">Logistic Regression:Sentiment Analysis</h2>

<b>Features: bag of 1-grams with TF-IDF values</b>:
- Extremely sparse feature matrix - close to 97% are zeros

 <b>Model: Logistic regression</b>
- $p(y = 1|x) = \sigma(w^{T}x)$
- Linear classification model
- Can handle sparse data
- Fast to train
- Weights can be interpreted
<img src="https://i.imgur.com/VieM41f.png" align="center" width=500 height=500>

### Task 1: Loading the dataset
---

In [15]:
import pandas as pd

df = pd.read_csv('D:\Sentiment Analysis Project\Project\movie_data.csv')     #loading dataset
df.head(10)             

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0
5,Leave it to Braik to put on a good show. Final...,1
6,Nathan Detroit (Frank Sinatra) is the manager ...,1
7,"To understand ""Crash Course"" in the right cont...",1
8,I've been impressed with Chavez's stance again...,1
9,This movie is directed by Renny Harlin the fin...,1


In [17]:
df['review'][0]

'In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />"Murder in Greenwich" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich famil

## <h2 align="center">Bag of words / Bag of N-grams model</h2>

### Task 2: Transforming documents into feature vectors

In [18]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count=CountVectorizer()





docs = np.array(['The sun is shining',
                'The weather is sweet',
                'The sun is shining, the weather is sweet, and one and one is two'])

bag=count.fit_transform(docs)

In [19]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [20]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


Raw term frequencies: *tf (t,d)*—the number of times a term t occurs in a document *d*

### Task 3: Word relevancy using term frequency-inverse document frequency

In [21]:
from sklearn.feature_extraction.text import TfidfTransformer
np.set_printoptions(precision=2)     ##just to make the matrix look cleaner
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)

print(tfidf.fit_transform(bag).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


### Task 4: Data Preparation

In [22]:
df.loc[0,'review'][-50:]  ##50 characters and above here we are looking

'is seven.<br /><br />Title (Brazil): Not Available'

In [23]:
import re
def preprocessor(text):    #below is the definition of pre processor
    text = re.sub('<[^>]*>', '', text)    #html tags
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)  #emoticons
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [24]:
preprocessor(df.loc[0,'review'][-50:])

'is seven title brazil not available'

In [25]:
preprocessor("</a>This :) is a :( test :-)!")   #notice the output. emojis removed from in between the text and moved to the end.

'this is a test :) :( :)'

In [26]:
df['review']=df['review'].apply(preprocessor)    #applying preprocessor to all reviews

### Task 5: Tokenization of documents

In [27]:
from nltk.stem.porter import PorterStemmer

porter=PorterStemmer()

In [28]:
def tokenizer(text):                 #function called tokenizer which takes text as an arguement
    return text.split()

In [29]:
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [30]:
tokenizer('Runners like running and thus they run.')  ##splits the sentence into words on the occurence of space

['Runners', 'like', 'running', 'and', 'thus', 'they', 'run.']

In [31]:
tokenizer_porter('Runners like running and thus they run')  #note, the ends of words have been stripped. lIke, runners is now run, running is now run.

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [32]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aadrit.aggarwal.in\AppData\Roaming\nltk_data.
[nltk_data]     ..
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner like running and runs a lot')[-10:] if w not in stop]
#above, we are first tokenizning and stemming our docuement, and then we are removing stop words like a, and etc

['runner', 'like', 'run', 'run', 'lot']

### Task 6: Transform Text Data into TF-IDF Vectors

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer  #simplifies the work for us, instead of doing it in several steps, it just does in one step
#here we are converting our original data into a feature matrix of tfidf values
tfidf=TfidfVectorizer(strip_accents=None,
                     lowercase=False,
                     preprocessor=None,                            #since we have already done preprocessing.
                     tokenizer=tokenizer_porter,
                     use_idf=True,
                     norm='l2',
                     smooth_idf=True)
y = df.sentiment.values       #target vector of sentiment values
X=tfidf.fit_transform(df.review)
                                                               

### Task 7: Document Classification using Logistic Regression

In [35]:
#first task is to divide to training and test sets using helper functions

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=1, test_size=0.5,
                                                shuffle = False)


In [36]:
import pickle

from sklearn.linear_model import LogisticRegressionCV  #this cv(cross validation) version helps to produce hyperparameters without manually loading them

clf = LogisticRegressionCV(cv=5,       #clf=classifier,cv=crossfold validations  
                          scoring='accuracy',
                          random_state=0,
                          n_jobs=-1,
                          verbose=3,
                          max_iter=300).fit(X_train,y_train)

saved_model = open('saved_model.sav', 'wb')      #wb is wriye byte to file
pickle.dump(clf,saved_model)
saved_model.close()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  1.8min remaining:  2.7min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.3min finished


### Task 8: Model Evaluation

In [37]:
filename = 'saved_model.sav'
saved_clf = pickle.load(open(filename,'rb'))  #rb is read byte

In [38]:
saved_clf.score(X_test, y_test)

C:\Users\aadrit.aggarwal.in\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:2260: ChangedBehaviorWarning: The long-standing behavior to use the accuracy score has changed. The scoring parameter is now used. This warning will disappear in version 0.22.
  ChangedBehaviorWarning)


0.89608